<h1> Climatological calculations from sounding data</h1>
<br>

This notebook computes daily climatological means from processed IGRA2 radiosonde data, creating a comprehensive atmospheric reference dataset for each day of the year. Using interpolated pressure-level data from individual soundings, the analysis generates climatological profiles that capture the typical atmospheric structure and variability patterns throughout the annual cycle.
#### Methodology
The climatological calculation employs a moving window approach to compute daily means, where each day of the year incorporates soundings from a specified temporal window (typically ±15 days) across all available years. This method ensures statistical robustness while preserving the seasonal signal in atmospheric variables. For each day, the process calculates ensemble means of fundamental atmospheric parameters (temperature, humidity, wind) and derives additional meteorological diagnostics including thermodynamic variables (potential temperature, equivalent potential temperature) and energy-related quantities (air density, wind power density).
<br><br>

In [1]:
# Modify the parameters in this cell according to your requirements

# ID of selected station
ID = 'COM00080222'

# Period of climatology
clim_start_year = 1991
clim_end_year = 2020
clim_hour = 12  # UTC hour of sonde release

# Path of the repository folder
path_repo = '/home/david/radiosonde_climatology_analysis/'

<br><br><br>
<br><br>

# 0. Preamble (functions and libs)

In [2]:
############
### folders' paths

docs_folder = f'{path_repo}/docs/'
station_folder = f'{path_repo}/data/{ID}/'
interpolated_folder = f'{station_folder}/interpolated/'
climatologies_folder = f'{station_folder}/climatology/'

In [3]:
import os
import shutil
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from metpy import calc as mpcalc
from metpy.units import units

def print_(str_to_print):
   datetime_now = datetime.now()
   print(datetime_now.strftime('[%Y-%m-%d %H:%M:%S]: ')+str_to_print)

def calculate_relative_humidity(temp, dew):
   temp = temp * units.degC
   dew = dew * units.degC    
   rh = mpcalc.relative_humidity_from_dewpoint(temp, dew)    
   rh = rh.magnitude * 100    
   return rh

def calculate_dewpoint(temp, rh):
   temp = temp * units.degC
   rh = rh * units.percent    
   dew = mpcalc.dewpoint_from_relative_humidity(temp, rh).to(units.degC)    
   dew = dew.magnitude
   return dew

def calculate_vapor_pressure(press, temp, dew):
   dew = dew * units.degC   
   temp = temp * units.degC
   press = press * units.hPa
   e = mpcalc.psychrometric_vapor_pressure_wet(press, temp, dew)    
   return e

def calculate_specific_humidity(press, dew):
   dew = dew * units.degC   
   press = press * units.hPa
   q = mpcalc.specific_humidity_from_dewpoint(press, dew).to('g/kg')  
   return q

def calculate_equivalent_potential_temperature(press, temp, dew):
   dew = dew * units.degC   
   temp = temp * units.degC
   press = press * units.hPa
   ept = mpcalc.equivalent_potential_temperature(press, temp, dew).to(units.degC)  
   return ept

def calculate_potential_temperature(press, temp):
   temp = temp * units.degC
   press = press * units.hPa
   pt = mpcalc.potential_temperature(press, temp).to(units.degC)  
   return pt

def calculate_wind_speed(u, v):
   return (u**2+v**2)**0.5

def calculate_air_density(press, temp, rh):
   return (press* 0.02897) / (8.31446*(1+(0.66*(rh/100.)*(temp+273.15)))) 

def calculate_air_density(press, temp, rh):
   temp = temp * units.degC
   press = press * units.hPa
   rh = rh /100. # relative humidity base 1
   mixing_ratio = mpcalc.mixing_ratio_from_relative_humidity(press, temp, rh).to('g/kg')
   rho = mpcalc.density(press, temp, mixing_ratio)
   return rho

def calculate_wind_power_density(rho, speed):
   return 0.5*rho*(speed**3)

############
### Creating the station's folders
os.makedirs(climatologies_folder, exist_ok = True)
os.makedirs(f'{climatologies_folder}/daily_mean', exist_ok = True)

<br><br><br>
<br><br>

# 1. Calculation of daily mean profiles

In [4]:
df_sondes = pd.read_csv(f'{interpolated_folder}/{ID}__availability.csv')
df_sondes = df_sondes.loc[(df_sondes['YEAR']>=clim_start_year)&(df_sondes['YEAR']<=clim_end_year)]
df_sondes = df_sondes.loc[df_sondes['HOUR']==clim_hour]
df_sondes['DATETIME'] = pd.to_datetime(df_sondes[['YEAR', 'MONTH', 'DAY', 'HOUR']])
df_sondes

,HEADREC,ID,YEAR,MONTH,DAY,HOUR,RELTIME,NUMLEV,P_SRC,NP_SRC,LAT,LON,DATETIME
1,#,COM00080222,1991,1,1,12,9999,40,ncdc6322,NaN,47000,-741500,1991-01-01 12:00:00
3,#,COM00080222,1991,1,2,12,9999,34,ncdc6322,NaN,47000,-741500,1991-01-02 12:00:00
6,#,COM00080222,1991,1,4,12,9999,37,usaf-ds3,NaN,47000,-741500,1991-01-04 12:00:00
8,#,COM00080222,1991,1,5,12,9999,28,ncdc6322,NaN,47000,-741500,1991-01-05 12:00:00
9,#,COM00080222,1991,1,6,12,9999,28,ncdc6322,NaN,47000,-741500,1991-01-06 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9161,#,COM00080222,2020,12,27,12,1139,179,ncdc-gts,ncdc-gts,47000,-741500,2020-12-27 12:00:00
9162,#,COM00080222,2020,12,28,12,1137,173,ncdc-gts,ncdc-gts,47000,-741500,2020-12-28 12:00:00
9163,#,COM00080222,2020,12,29,12,1134,156,ncdc-gts,ncdc-gts,47000,-741500,2020-12-29 12:00:00
9165,#,COM00080222,2020,12,30,12,1131,126,ncdc-gts,ncdc-gts,47000,-741500,2020-12-30 12:00:00


In [5]:
print_(f'Calculating daily climatological means for {ID}')
print_('It could take several minutes...')

# Moving window approach to compute daily means
window = 15  # days

# Review of available days
datetime_start_i = datetime(clim_start_year, 1, 1, 12)
datetime_end_i = datetime(clim_end_year, 12, 31, 12)

# Pre-load all sounding files into memory (if feasible) or create a file cache
print_('Pre-loading sounding data...')
sounding_cache = {}
available_files = set()

# Get list of available files
import glob
available_file_paths = glob.glob(f'{interpolated_folder}/{ID}_*.csv')
for file_path in available_file_paths:
    filename = os.path.basename(file_path)
    file_date_str = filename.split('_')[1].replace('.csv', '')
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d%H%M')
        available_files.add(file_date)
    except:
        continue

print_(f'Found {len(available_files)} available sounding files')

# Pre-filter df_sondes to only include dates within our climatology period
df_sondes_filtered = df_sondes[
    (df_sondes['DATETIME'] >= datetime_start_i) & 
    (df_sondes['DATETIME'] <= datetime_end_i) & 
    (df_sondes['DATETIME'].isin(available_files))
].copy()

print_(f'Filtered to {len(df_sondes_filtered)} soundings within climatology period')

# Function to calculate derived variables from means
def calculate_derived_from_means(df_means):
    """Calculate derived atmospheric variables from climatological means"""
    # Calculate derived variables from the averaged primary variables
    df_means['dewpoint_temperature'] = calculate_dewpoint(df_means['temperature'].values, df_means['relative_humidity'].values)
    df_means['vapor_pressure'] = calculate_vapor_pressure(df_means['pressure'].values, df_means['temperature'].values, df_means['dewpoint_temperature'].values)
    df_means['specific_humidity'] = calculate_specific_humidity(df_means['pressure'].values, df_means['dewpoint_temperature'].values)
    df_means['equivalent_potential_temperature'] = calculate_equivalent_potential_temperature(df_means['pressure'].values, df_means['temperature'].values, df_means['dewpoint_temperature'].values)
    df_means['potential_temperature'] = calculate_potential_temperature(df_means['pressure'].values, df_means['temperature'].values)
    df_means['wind_speed'] = calculate_wind_speed(df_means['u_wind'].values, df_means['v_wind'].values)
    df_means['air_density'] = calculate_air_density(df_means['pressure'].values, df_means['temperature'].values, df_means['relative_humidity'].values)
    df_means['wind_power_density'] = calculate_wind_power_density(df_means['air_density'].values, df_means['wind_speed'].values)
    return df_means

# Batch process soundings for efficiency (only primary variables)
def process_soundings_batch(file_list, batch_size=50):
    """Process multiple soundings in batches - primary variables only"""
    all_soundings = []
    
    for i in range(0, len(file_list), batch_size):
        batch_files = file_list[i:i+batch_size]
        batch_dfs = []
        
        # Read batch of files
        for filename in batch_files:
            try:
                df_temp = pd.read_csv(f'{interpolated_folder}/{filename}')
                # Rename columns and keep only primary variables
                df_temp = df_temp.rename(columns={
                    'PRESS': 'pressure',
                    'GPH': 'geopotential_height', 
                    'TEMP': 'temperature',
                    'RH': 'relative_humidity',
                    'U': 'u_wind',
                    'V': 'v_wind'
                })
                # Keep only primary variables for averaging
                primary_vars = ['pressure', 'geopotential_height', 'temperature', 'relative_humidity', 'u_wind', 'v_wind']
                df_temp = df_temp[primary_vars]
                batch_dfs.append(df_temp)
            except Exception as e:
                continue
        
        if batch_dfs:
            # Concatenate batch (no derived calculations yet)
            batch_combined = pd.concat(batch_dfs, ignore_index=True)
            all_soundings.append(batch_combined)
    
    return pd.concat(all_soundings, ignore_index=True) if all_soundings else pd.DataFrame()

# Initialize progress tracking variables
total_days = 365
completed_days = 0
last_printed_percentage = 0

# Standard pressure levels
standard_pressures = list(range(1010, 95, -5))
column_names = ['pressure', 'geopotential_height', 'temperature', 'relative_humidity', 'u_wind', 'v_wind', 
                'dewpoint_temperature', 'vapor_pressure', 'specific_humidity', 'equivalent_potential_temperature', 
                'potential_temperature', 'wind_speed', 'air_density', 'wind_power_density']

for dayofyear_i in range(1, 366):  # Include day 365
    # Select soundings for the climatological mean for that day of year
    target_dates = []
    for year_i in range(datetime_start_i.year, datetime_end_i.year + 1):
        base_date = datetime(year_i, 1, 1) + timedelta(days=dayofyear_i - 1, hours=clim_hour)
        window_dates = [base_date + timedelta(days=dt) for dt in range(-window, window)]
        target_dates.extend(window_dates)
    
    # Filter available soundings for this day of year (much faster with pre-filtered data)
    df_sondes_date_i = df_sondes_filtered[df_sondes_filtered['DATETIME'].isin(target_dates)]
    
    # Check available information
    # At least 40% of the data. I set this threshold for greater flexibility, but feel free to adjust it if you consider it necessary
    if len(df_sondes_date_i) < len(target_dates) * 0.4:
        print_(f'Not enough information available for day of year {dayofyear_i:03d}.')
        # Export blank dataframe
        df_mean_i = pd.DataFrame(columns=column_names)
        for press_i in standard_pressures:
            df_mean_i.loc[len(df_mean_i), 'pressure'] = press_i
        df_mean_i.to_csv(f'{climatologies_folder}/daily_mean/soundingmean_day_{dayofyear_i:03d}.csv', index=False)
        
        # Update progress
        completed_days += 1
        current_percentage = (completed_days * 100) // total_days
        if current_percentage >= last_printed_percentage + 10 and current_percentage <= 100:
            print_(f'Progress: {current_percentage}% ({completed_days}/{total_days} days)')
            last_printed_percentage = current_percentage
        continue
    
    # Create list of filenames to process
    file_list = []
    for _, row in df_sondes_date_i.iterrows():
        filename = f'{ID}_{row["DATETIME"].strftime("%Y%m%d%H%M")}.csv'
        file_list.append(filename)
    
    # Process soundings in batches (primary variables only)
    df_combined = process_soundings_batch(file_list)
    
    if df_combined.empty:
        print_(f'No valid data processed for day of year {dayofyear_i:03d}.')
        # Export blank dataframe
        df_mean_i = pd.DataFrame(columns=column_names)
        for press_i in standard_pressures:
            df_mean_i.loc[len(df_mean_i), 'pressure'] = press_i
        df_mean_i.to_csv(f'{climatologies_folder}/daily_mean/soundingmean_day_{dayofyear_i:03d}.csv', index=False)
    else:
        # Calculate climatological means for primary variables only
        primary_columns = ['geopotential_height', 'temperature', 'relative_humidity', 'u_wind', 'v_wind']
        df_mean_i = df_combined.groupby('pressure')[primary_columns].mean().reset_index()
        df_mean_i = df_mean_i.sort_values('pressure', ascending=False)
        
        # Now calculate derived variables from the means
        df_mean_i = calculate_derived_from_means(df_mean_i)
        
        df_mean_i.to_csv(f'{climatologies_folder}/daily_mean/soundingmean_day_{dayofyear_i:03d}.csv', index=False)
    
    # Update progress
    completed_days += 1
    current_percentage = (completed_days * 100) // total_days
    if current_percentage >= last_printed_percentage + 10 and current_percentage <= 100:
        print_(f'Progress: {current_percentage}% ({completed_days}/{total_days} days)')
        last_printed_percentage = current_percentage

print_(f'Climatological daily means calculation completed for {ID}')

[2025-07-28 11:34:30]: Calculating daily climatological means for COM00080222
[2025-07-28 11:34:30]: It could take several minutes...
[2025-07-28 11:34:30]: Pre-loading sounding data...
[2025-07-28 11:34:30]: Found 9168 available sounding files
[2025-07-28 11:34:30]: Filtered to 8630 soundings within climatology period
[2025-07-28 11:34:56]: Progress: 10% (37/365 days)
[2025-07-28 11:35:22]: Progress: 20% (73/365 days)
[2025-07-28 11:35:48]: Progress: 30% (110/365 days)
[2025-07-28 11:36:14]: Progress: 40% (146/365 days)
[2025-07-28 11:36:42]: Progress: 50% (183/365 days)
[2025-07-28 11:37:09]: Progress: 60% (219/365 days)
[2025-07-28 11:37:36]: Progress: 70% (256/365 days)
[2025-07-28 11:38:02]: Progress: 80% (292/365 days)
[2025-07-28 11:38:30]: Progress: 90% (329/365 days)
[2025-07-28 11:38:58]: Progress: 100% (365/365 days)
[2025-07-28 11:38:58]: Climatological daily means calculation completed for COM00080222


<br><br><br>
<br><br>

# 2. Annual cycle dataset arrangement
This section rearranges the daily climatological means into variable-specific annual cycle datasets where each row represents a pressure level and each column represents a day of the year (1-365). This format facilitates analysis of seasonal patterns, annual cycles, and temporal variability for each atmospheric variable throughout the year.

In [6]:
variables = ['geopotential_height', 'temperature', 'relative_humidity',
             'u_wind', 'v_wind', 'dewpoint_temperature', 'vapor_pressure',
             'specific_humidity', 'equivalent_potential_temperature',
             'potential_temperature', 'wind_speed', 'air_density',
             'wind_power_density']

for variable in variables:
    # Pre-load all daily mean files once
    daily_data = {}
    for dayofyear_i in range(1, 366):
        df_mean_i = pd.read_csv(f'{climatologies_folder}/daily_mean/soundingmean_day_{dayofyear_i:03d}.csv')
        daily_data[dayofyear_i] = df_mean_i[variable]
    
    # Create DataFrame with all columns at once
    df_var = pd.DataFrame(daily_data)
    df_var.insert(0, 'pressure', list(range(1010, 95, -5)))
    
    df_var.to_csv(f'{climatologies_folder}/{variable}.csv', index=False)

print_(f'Annual cycle datasets arranged for all variables at {ID}')

[2025-07-28 11:39:03]: Annual cycle datasets arranged for all variables at COM00080222
